**Import libraries**

In [1]:
!pip install https://github.com/ElsevierDev/elsapy/archive/master.zip

  Using cached https://github.com/ElsevierDev/elsapy/archive/master.zip


In [2]:
#xml.etree.ElementTree — The ElementTree XML API
import xml.etree.ElementTree as ET

In [3]:
#Import some basic libraries
import numpy as np
import pandas as pd
from pprint import pprint

In [4]:
#Some important elsevier api libraries
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
import csv
import requests
import xmltodict
import urllib3
import re

#Import warnings library
import warnings
warnings.filterwarnings("ignore")

In [5]:
#config = {'apikey': '22d18e0c4fdc59bb8898844056b2be57'}
config = {
    "apikey": "2dc442325fc67f2f275ec3157ef8df65",
 "insttoken": "6beb1f6c29d85f50029bf11c8de94d1b"
    }

client = ElsClient(config['apikey'])
client.inst_token = config['insttoken']

### Science Direct Full Text API

In [8]:
#The data loaded here with links
df = pd.read_csv(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\doi_6000_withlink.csv')
df.head()

,URL,Title,Pub_Name,DOI,Link
0,https://api.elsevier.com/content/article/pii/S...,Homogeneous first-row transition metal catalys...,Tetrahedron,10.1016/j.tet.2021.132473,https://api.elsevier.com/content/article/doi/1...
1,https://api.elsevier.com/content/article/pii/S...,Selective photoelectrocatalytic tuning of benz...,Applied Catalysis B: Environmental,10.1016/j.apcatb.2020.119868,https://api.elsevier.com/content/article/doi/1...
2,https://api.elsevier.com/content/article/pii/S...,Light alcohols reforming towards renewable hyd...,Renewable and Sustainable Energy Reviews,10.1016/j.rser.2020.110523,https://api.elsevier.com/content/article/doi/1...
3,https://api.elsevier.com/content/article/pii/S...,Minimized Pt deposition on CdS simultaneously ...,Applied Surface Science,10.1016/j.apsusc.2021.150446,https://api.elsevier.com/content/article/doi/1...
4,https://api.elsevier.com/content/article/pii/S...,Boosting photocatalytic hydrogen production co...,Chemical Engineering Journal,10.1016/j.cej.2021.129870,https://api.elsevier.com/content/article/doi/1...


**Random check the links** 

In [9]:
#defining a header's dictionary to pass through requests
headers_dict = {"X-ELS-APIKey": config['apikey'], "X-ELS-Insttoken": config['insttoken'], "Accept": "application/xml"}

yes = 0
no = 0
for i in range(0, df.shape[0]):
    #x takes response of the HTTP request, passes link
    x = requests.get(df['Link'][i], headers=headers_dict)
    #print(x.text) #check
        
    #saving it as XML file
    with open("full_text.xml", 'wb') as f:
        f.write(x.content)
        
    tree = ET.parse(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\full_text.xml')
    root = tree.getroot()
    
    original_text = root.find('{http://www.elsevier.com/xml/svapi/article/dtd}originalText')
    doc = original_text.find('{http://www.elsevier.com/xml/xocs/dtd}doc')
    serial_item = doc.find('{http://www.elsevier.com/xml/xocs/dtd}serial-item')
    print(serial_item)

    if serial_item != None:
        yes +=1
        print(yes)
    else:
        #print("Full text for this paper doesn't exist")
        no+=1
        
print(yes)
print(no)

<Element '{http://www.elsevier.com/xml/xocs/dtd}serial-item' at 0x00000269564789A0>
1
<Element '{http://www.elsevier.com/xml/xocs/dtd}serial-item' at 0x0000026956665720>
2
<Element '{http://www.elsevier.com/xml/xocs/dtd}serial-item' at 0x00000269564FC7C0>
3
<Element '{http://www.elsevier.com/xml/xocs/dtd}serial-item' at 0x000002695659B860>
4
4
0


### Code to Extract Coredata

In [11]:
#Coredata is present in every paper

#Element.findall() finds only elements with a tag which are direct children of the current element
#Element.find() finds the first child with a particular tag

#defining a header's dictionary to pass through requests
headers_dict = {"X-ELS-APIKey": config['apikey'], "X-ELS-Insttoken": config['insttoken'], "Accept": "application/xml"}

#x takes response of the HTTP request, random passes link
x = requests.get(df['Link'][3], headers=headers_dict)
#print(x.text) #check
    
#saving it as XML file
with open("full_text.xml", 'wb') as f:
    f.write(x.content)

#Reading the data
#<?xml version="1.0" encoding="UTF-8"?>
tree = ET.parse(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\full_text.xml')
root = tree.getroot()

for entry in root.findall('{http://www.elsevier.com/xml/svapi/article/dtd}coredata'):
    
    url = []; doi =[]; title = []; pub_name =[]; type_ = []
    description = []
    #CHECK CODE
    url.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text)
    doi.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text)
    title.append(entry.find('{http://purl.org/dc/elements/1.1/}title').text)
    pub_name.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text)
    type_.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}aggregationType').text)
    description.append(entry.find('{http://purl.org/dc/elements/1.1/}description').text)
    description[0] = " ".join(description[0].split())
    print('URL:',url )
    print('DOI:', doi)
    print('Title:', title)
    print('Journal name:',pub_name )
    print('Type:',type_ )
    print('\n')
    print('Abstract:',description[0] )
    print('\n')
    
#Extract the list of authors and keywords mentioned      
original_text = root.find('{http://www.elsevier.com/xml/svapi/article/dtd}originalText')
#print(originaltext)
doc = original_text.find('{http://www.elsevier.com/xml/xocs/dtd}doc')
#print(doc)
serial_item = doc.find('{http://www.elsevier.com/xml/xocs/dtd}serial-item')
#print(serial_item)

keyword_list = []
author_list = []
if serial_item != None:
    article = serial_item.find('{http://www.elsevier.com/xml/ja/dtd}article')
    #print(article)
    head = article.find('{http://www.elsevier.com/xml/ja/dtd}head')
    #print(head)
    author_group = head.find('{http://www.elsevier.com/xml/common/dtd}author-group')
    #print(author_group)

    keywords_ = head.find('{http://www.elsevier.com/xml/common/dtd}keywords')
    #print(keywords_)

    for author in author_group.findall('{http://www.elsevier.com/xml/common/dtd}author'):
        name = author.find('{http://www.elsevier.com/xml/common/dtd}given-name').text
        surname = author.find('{http://www.elsevier.com/xml/common/dtd}surname').text
        author_list.append(name + ' ' + surname)
        #print('\n')

    for word in keywords_.itertext():
        keyword_list.append(word)

    keyword_list = "".join(keyword_list)
    keyword_list = list(keyword_list.split())
    #keywords, authors present in the paper
    print('Keywords present:',keyword_list)
    print('Authors: ',author_list)
else:
    print("Full text for this paper doesn't exist")

URL: ['https://api.elsevier.com/content/article/pii/S0169433221023370']
DOI: ['10.1016/j.apsusc.2021.151284']
Title: ['CdS(ZB)/CdS(WZ)/Ni-BTC photocatalytic selective oxidation of benzyl alcohol to benzaldehyde coupled with hydrogen evolution ']
Journal name: ['Applied Surface Science']
Type: ['Journal']


Abstract: Development of difunctional photocatalyst for both hydrogen production and high value-added fine chemicals is of great industrial significance. Cubic crystal phase zinc blende CdS(ZB) and hexagonal crystal phase wurtzite CdS(WZ) in CdS(ZB)/CdS(WZ)/Ni-BTC (BTC = 1,3,5-benzene tricarboxylic acid) composites can form intergrowth homojunction to improve the photoelectron-hole separation efficiency, so that the composites show excellent catalytic activity in photocatalytic selective oxidation of benzyl alcohol to benzaldehyde coupled with hydrogen evolution reaction. The conversion of benzyl alcohol can up to 99.0%, the yield of benzaldehyde is 90.2%, and the yield of hydrogen i

### Code to extract full text and store in csv

In [14]:
#defining a header's dictionary to pass through requests
headers_dict = {"X-ELS-APIKey": "2dc442325fc67f2f275ec3157ef8df65", "X-ELS-Insttoken": "6beb1f6c29d85f50029bf11c8de94d1b", "Accept": "application/xml"}
df_temp = pd.DataFrame()
excel_writer = pd.ExcelWriter(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\test2.xlsx', engine='xlsxwriter')
for i in np.arange(0,2,1):
    print(i, 'done' )
    #x takes response of the HTTP request, passes link
    x = requests.get(df['Link'][i], headers=headers_dict)

    #saving it as XML file
    with open("full_text.xml", 'wb') as f:
        f.write(x.content)

    #Reading the data
    #<?xml version="1.0" encoding="UTF-8"?>
    tree = ET.parse(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\full_text.xml')
    root = tree.getroot()
    original_text = root.find('{http://www.elsevier.com/xml/svapi/article/dtd}originalText')
    
    if original_text != None:
        doc = original_text.find('{http://www.elsevier.com/xml/xocs/dtd}doc')
        serial_item = doc.find('{http://www.elsevier.com/xml/xocs/dtd}serial-item')

        if serial_item != None:
            article = serial_item.find('{http://www.elsevier.com/xml/ja/dtd}article')
            
            if article != None:
                body = article.find('{http://www.elsevier.com/xml/ja/dtd}body')
                
                if body != None:
                    sections = body.find('{http://www.elsevier.com/xml/common/dtd}sections')
                    
                    #lists for dataframe
                    label_list = [] 
                    title_list = []
                    para_list = []

                    for section1 in sections.findall('{http://www.elsevier.com/xml/common/dtd}section'):
                        if section1.find('{http://www.elsevier.com/xml/common/dtd}label') != None:
                        #if section1 != None:
                            label = section1.find('{http://www.elsevier.com/xml/common/dtd}label').text
                            label_list.append(label)
                            section_title_list = []
                            section_title = section1.find('{http://www.elsevier.com/xml/common/dtd}section-title')
                            for sec in section_title.itertext():
                                section_title_list.append(sec)

                            section_title_list = "".join(section_title_list)
                            section_title_list = list(section_title_list.split()) 
                            section_title_list = " ".join(section_title_list)
                            title_list.append(section_title_list)
                            print(section_title_list)

                            total_para = []
                            for paragraph in section1.findall('{http://www.elsevier.com/xml/common/dtd}para'):
                                if paragraph != None:
                                    paragraph1 = []
                                    for p in paragraph.itertext():
                                        paragraph1.append(p)

                                    paragraph1 = "".join(paragraph1)
                                    paragraph1 = list(paragraph1.split()) 
                                    paragraph1 = " ".join(paragraph1)
                                    total_para.append(paragraph1)

                            total_para = " ".join(total_para)
                            #print(total_para)
                            para_list.append(total_para)
                            print('\n')

                            for section2 in section1.findall('{http://www.elsevier.com/xml/common/dtd}section'):
                                if section2.find('{http://www.elsevier.com/xml/common/dtd}label') !=None:

                                    label = section2.find('{http://www.elsevier.com/xml/common/dtd}label').text

                                    label_list.append(label)
                                    print('sub-section ', label)

                                    sub_section_title_list = []

                                    sub_section_title = section2.find('{http://www.elsevier.com/xml/common/dtd}section-title')
                                    for sub in sub_section_title.itertext():
                                        sub_section_title_list.append(sub)

                                    sub_section_title_list = "".join(sub_section_title_list)
                                    sub_section_title_list = list(sub_section_title_list.split()) 
                                    sub_section_title_list = " ".join(sub_section_title_list)
                                    title_list.append(sub_section_title_list)
                                    #print('sub-section-title: ', sub_section_title_list)

                                    para1 = []
                                    for para in section2.itertext():
                                    #findall('{http://www.elsevier.com/xml/common/dtd}para'):
                                        #paragraph = para.
                                        #for i in paragraph: 
                                        para1.append(para)

                                    para1 = "".join(para1)
                                    para1 = list(para1.split()) 
                                    para1 = " ".join(para1)
                                    #print(para1) 
                                    para_list.append(para1)
                                    print('\n')

    temp = dict(doi = np.array(doi), 
         title = np.array(title), 
         pub_name = np.array(pub_name), 
         Type = np.array(type_), 
         abstract= np.array(description), 
         authors= np.array(author_list), 
         keyword =np.array(keyword_list),
         title_label = np.array(label_list),
         titles = np.array(title_list),
         text = np.array(para_list))
    
    #print(len(temp['text']), type(temp['text']))
    #print(len(temp['titles']), type(temp['titles']))
    
    df_temp = pd.DataFrame({'sections' : temp['titles'].tolist(), 'Text': temp['text'].tolist()})
    
    #df_temp.head()
    
    #Save each data frame
    df_temp.to_csv(r'{}.csv'.format(i))

0 done
Introduction


Results and discussion


sub-section  2.1


sub-section  2.2


sub-section  2.3


Conclusion


1 done
Introduction


Results and discussion


sub-section  2.1


sub-section  2.2


sub-section  2.3


Conclusion




### Extract the abstract

In [10]:
%%time
list_abstract = []
list_doi= []
list_title = []

## ScienceDirect (full-text) document example using DOI
#for i in range(len(data)):
from pprint import pprint
doi= df['DOI']
for i in range(df.shape[0]):
    pprint(doi[i])
    doi_doc = FullDoc(doi = doi[i])
    if doi_doc.read(client):

        #pprint((doi_doc.title))
        abstract = doi_doc._data['coredata']['dc:description']
        title = doi_doc.title
        #pprint(abstract)
        print(i, 'done')
        #dict_abstract['DOI'] = doi[i]
        #dict_abstract['Abstract'] = abstract
        list_abstract.append(abstract)
        list_doi.append(doi[i])
        list_title.append(title)
        #doi_doc.write()

    else:
        pprint('Operation failed')
        #dict_abstract['DOI'] = doi[i]
        #dict_abstract['Abstract'] = 'NAN'
        list_abstract.append('0')
        list_doi.append('1')
        list_title.append('2')

'10.1016/j.tet.2021.132473'
0 done
'10.1016/j.apcatb.2020.119868'
1 done
'10.1016/j.rser.2020.110523'
2 done
'10.1016/j.apsusc.2021.150446'
3 done
'10.1016/j.cej.2021.129870'
4 done
'10.1016/B978-0-12-823827-1.00001-8'
5 done
'10.1016/j.mcat.2020.110912'
6 done
'10.1016/j.jcis.2021.02.036'
7 done
'10.1016/j.apsusc.2021.151284'
8 done
'10.1016/j.jelechem.2021.115820'
9 done
'10.1016/j.ica.2021.120488'
10 done
'10.1016/j.apcata.2020.117703'
11 done
'10.1016/j.biortech.2020.124168'
12 done
'10.1016/j.chemosphere.2021.131558'
13 done
'10.1016/j.ijhydene.2019.10.114'
14 done
'10.1016/j.cclet.2019.10.001'
15 done
'10.1016/j.ijhydene.2019.11.166'
16 done
'10.1016/j.ica.2021.120245'
17 done
'10.1016/j.jclepro.2021.128546'
18 done
'10.1016/j.apsusc.2021.151506'
19 done
'10.1016/j.cej.2021.133211'
20 done
'10.1016/j.apcatb.2021.120556'
21 done
'10.1016/j.jcat.2021.08.038'
22 done
'10.1016/j.jct.2021.106591'
23 done
'10.1016/j.coelec.2021.100841'
24 done
'10.1016/j.molliq.2021.117807'
25 done
'10

208 done
'10.1016/j.chemosphere.2021.132548'
209 done
'10.1016/j.cclet.2021.06.082'
210 done
'10.1016/j.cej.2021.131673'
211 done
'10.1016/j.cattod.2019.08.045'
212 done
'10.1016/j.apsusc.2020.146983'
213 done
'10.1016/j.jaap.2021.105111'
214 done
'10.1016/j.ijhydene.2010.12.013'
215 done
'10.1016/j.matpr.2021.10.012'
216 done
'10.1016/j.ijhydene.2021.03.142'
217 done
'10.1016/j.ijhydene.2017.02.195'
218 done
'10.1016/j.ijhydene.2021.05.035'
219 done
'10.1016/j.ijhydene.2021.04.062'
220 done
'10.1016/j.ijhydene.2020.07.160'
221 done
'10.1016/j.ijhydene.2021.09.139'
222 done
'10.1016/j.ijhydene.2019.06.115'
223 done
'10.1016/j.ijhydene.2021.04.146'
224 done
'10.1016/j.ijhydene.2021.11.041'
225 done
'10.1016/j.apcatb.2021.120648'
226 done
'10.1016/j.jallcom.2021.162168'
227 done
'10.1016/j.ijhydene.2015.06.003'
228 done
'10.1016/j.xcrp.2021.100627'
229 done
'10.1016/j.ijhydene.2020.11.244'
230 done
'10.1016/j.ijhydene.2020.07.233'
231 done
'10.1016/j.apcata.2016.01.007'
232 done
'10.1016

411 done
'10.1016/j.apcatb.2010.10.009'
412 done
'10.1016/j.ijhydene.2020.09.012'
413 done
'10.1016/j.coche.2020.05.009'
414 done
'10.1016/j.ijhydene.2020.04.207'
415 done
'10.1016/j.molliq.2021.117188'
416 done
'10.1016/j.apcata.2016.05.022'
417 done
'10.1016/j.biortech.2020.124370'
418 done
'10.1016/j.ijhydene.2020.03.002'
419 done
'10.1016/j.ijhydene.2020.05.139'
420 done
'10.1016/j.apcata.2021.118266'
421 done
'10.1016/j.carres.2021.108341'
422 done
'10.1016/j.electacta.2021.138335'
423 done
'10.1016/j.ijhydene.2020.05.238'
424 done
'10.1016/j.jelechem.2017.09.021'
425 done
'10.1016/j.cattod.2016.05.023'
426 done
'10.1016/j.ijhydene.2020.05.165'
427 done
'10.1016/j.supflu.2021.105329'
428 done
'10.1016/j.fuproc.2019.106193'
429 done
'10.1016/j.jechem.2021.03.022'
430 done
'10.1016/j.ijhydene.2019.11.127'
431 done
'10.1016/j.fuel.2021.120457'
432 done
'10.1016/j.ijhydene.2021.01.004'
433 done
'10.1016/j.chemosphere.2021.131360'
434 done
'10.1016/j.jpcs.2021.110228'
435 done
'10.1016

613 done
'10.1016/j.apcatb.2020.119759'
614 done
'10.1016/j.poly.2021.115579'
615 done
'10.1016/j.cej.2020.126721'
616 done
'10.1016/j.ijhydene.2020.03.217'
617 done
'10.1016/S1872-5813(16)30009-3'
618 done
'10.1016/j.biortech.2020.124169'
619 done
'10.1016/j.ijhydene.2019.09.212'
620 done
'10.1016/j.jcat.2016.01.012'
621 done
'10.1016/j.jechem.2020.03.054'
622 done
'10.1016/j.enconman.2021.114341'
623 done
'10.1016/j.matpr.2021.09.162'
624 done
'10.1016/j.ijhydene.2020.10.128'
625 done
'10.1016/j.ijhydene.2019.04.270'
626 done
'10.1016/j.ijhydene.2018.09.048'
627 done
'10.1016/j.catcom.2011.07.006'
628 done
'10.1016/j.catcom.2011.05.024'
629 done
'10.1016/j.freeradbiomed.2017.04.344'
630 done
'10.1016/j.biombioe.2020.105712'
631 done
'10.1016/j.tet.2014.10.007'
632 done
'10.1016/j.surfin.2021.100979'
633 done
'10.1016/j.jcat.2012.05.007'
634 done
'10.1016/j.jiec.2014.11.013'
635 done
'10.1016/j.ijhydene.2019.09.078'
636 done
'10.1016/j.ijhydene.2021.09.115'
637 done
'10.1016/j.fuproc.

815 done
'10.1016/j.ijhydene.2020.07.205'
816 done
'10.1016/j.jbiosc.2018.10.008'
817 done
'10.1016/j.apcatb.2021.120120'
818 done
'10.1016/j.materresbull.2020.110872'
819 done
'10.1016/j.rser.2020.110338'
820 done
'10.1016/j.ijhydene.2019.06.179'
821 done
'10.1016/j.matpr.2017.09.079'
822 done
'10.1016/j.ces.2007.03.007'
823 done
'10.1016/j.apcata.2018.07.002'
824 done
'10.1016/j.bbagen.2020.129681'
825 done
'10.1016/j.cogsc.2020.100386'
826 done
'10.1016/j.chemosphere.2021.131174'
827 done
'10.1016/j.fuel.2021.122401'
828 done
'10.1016/j.cattod.2012.05.004'
829 done
'10.1016/j.biortech.2017.07.044'
830 done
'10.1016/j.ijhydene.2019.06.050'
831 done
'10.1016/j.jclepro.2019.04.365'
832 done
'10.1016/j.fuproc.2016.03.015'
833 done
'10.1016/j.ijhydene.2019.01.246'
834 done
'10.1016/j.ijhydene.2019.03.018'
835 done
'10.1016/j.jpcs.2020.109457'
836 done
'10.1016/j.ijhydene.2019.01.045'
837 done
'10.1016/B978-0-12-817110-3.00003-5'
838 done
'10.1016/j.ijhydene.2018.07.184'
839 done
'10.1016

...

4966 done
'10.1016/j.fuel.2016.09.026'
4967 done
'10.1016/j.indcrop.2019.111695'
4968 done
'10.1016/j.jpowsour.2007.03.084'
4969 done
'10.1016/j.jpowsour.2020.228860'
4970 done
'10.1016/j.jcis.2020.01.115'
4971 done
'10.1016/j.jcomc.2021.100109'
4972 done
'10.1016/j.catcom.2015.01.005'
4973 done
'10.1016/j.ijhydene.2020.07.260'
4974 done
'10.1016/j.cej.2020.125113'
4975 done
'10.1016/j.jcis.2021.05.042'
4976 done
'10.1074/jbc.M112.429985'
4977 done
'10.1016/j.egypro.2017.12.121'
4978 done
'10.1016/j.jcis.2019.11.037'
4979 done
'10.1016/j.fuel.2019.116677'
4980 done
'10.1016/j.cattod.2005.06.036'
4981 done
'10.1016/j.ijhydene.2009.02.011'
4982 done
'10.1016/j.ijhydene.2018.02.107'
4983 done
'10.1016/j.jcat.2014.11.013'
4984 done
'10.1016/j.apcata.2003.07.013'
4985 done
'10.1016/j.biopha.2017.11.025'
4986 done
'10.1016/j.procbio.2007.11.007'
4987 done
'10.1016/j.jphotochem.2020.112890'
4988 done
'10.1016/j.jallcom.2020.154353'
4989 done
'10.1016/j.jcat.2019.05.033'
4990 done
'10.1016/j.i

5163 done
'10.1016/j.jiec.2019.09.007'
5164 done
'10.1016/j.eurpolymj.2019.109460'
5165 done
'10.1016/j.biteb.2021.100677'
5166 done
'10.1016/j.biortech.2010.12.106'
5167 done
'10.1016/j.molliq.2018.04.058'
5168 done
'10.1016/j.micromeso.2018.06.003'
5169 done
'10.1016/S0378-3820(02)00020-6'
5170 done
'10.1016/j.jece.2015.08.021'
5171 done
'10.1016/j.fpsl.2019.100445'
5172 done
'10.1016/j.fuel.2018.05.075'
5173 done
'10.1016/B978-0-08-102688-5.00056-8'
5174 done
'10.1016/j.cej.2020.125750'
5175 done
'10.1016/j.catcom.2018.02.010'
5176 done
'10.1016/j.cej.2020.126409'
5177 done
'10.1016/j.jmrt.2019.09.074'
5178 done
'10.1016/j.jcat.2008.10.012'
5179 done
'10.1016/j.cej.2021.131524'
5180 done
'10.1016/j.polymer.2011.11.029'
5181 done
'10.1016/j.apsusc.2013.04.122'
5182 done
'10.1016/j.catcom.2013.07.012'
5183 done
'10.1016/j.ijhydene.2018.09.109'
5184 done
'10.1016/j.ijhydene.2016.08.034'
5185 done
'10.1016/j.mcat.2017.12.032'
5186 done
'10.1016/j.apsusc.2019.143808'
5187 done
'10.1016/j

5362 done
'10.1016/S1389-1723(01)80112-2'
5363 done
'10.1016/j.indcrop.2019.05.025'
5364 done
'10.1016/j.apcata.2015.01.046'
5365 done
'10.1016/j.apsusc.2019.03.035'
5366 done
'10.1016/j.envres.2020.109563'
5367 done
'10.1016/j.jphotochem.2009.02.001'
5368 done
'10.1016/j.molstruc.2021.131294'
5369 done
'10.1016/j.jchromb.2014.09.004'
5370 done
'10.1016/j.apcatb.2019.02.012'
5371 done
'10.1016/j.jbiosc.2019.07.001'
5372 done
'10.1016/j.alcohol.2016.08.005'
5373 done
'10.1016/j.apcatb.2021.120522'
5374 done
'10.1016/j.cattod.2018.03.038'
5375 done
'10.1016/j.jhazmat.2018.09.006'
5376 done
'10.1016/j.molcata.2014.04.022'
5377 done
'10.1016/j.fuel.2019.116336'
5378 done
'10.1016/j.compositesa.2019.105720'
5379 done
'10.1016/0926-860X(93)80211-8'
5380 done
'10.1016/j.ica.2020.119808'
5381 done
'10.1016/j.cej.2020.125470'
5382 done
'10.1016/j.neuint.2014.01.004'
5383 done
'10.1016/j.jcat.2019.10.024'
5384 done
'10.1016/j.mcat.2019.110551'
5385 done
'10.1016/j.snb.2020.129281'
5386 done
'10.

5561 done
'10.1016/j.carbpol.2020.116111'
5562 done
'10.1016/j.ijbiomac.2019.02.052'
5563 done
'10.1016/j.ijhydene.2012.04.077'
5564 done
'10.1016/B978-0-12-821728-3.00001-3'
5565 done
'10.1016/j.jwpe.2020.101718'
5566 done
'10.1016/j.wasman.2021.04.020'
5567 done
'10.1016/j.cej.2015.12.006'
5568 done
'10.1016/j.ijhydene.2008.02.055'
5569 done
'10.1016/j.msea.2010.09.100'
5570 done
'10.1016/j.molstruc.2021.131283'
5571 done
'10.1016/B978-008044704-9/50106-3'
5572 done
'10.1016/j.intermet.2020.106797'
5573 done
'10.1016/j.ica.2008.05.017'
5574 done
'10.1016/B978-1-78242-361-4.00011-X'
5575 done
'10.1016/j.jep.2018.06.008'
5576 done
'10.1016/j.foodres.2018.08.008'
5577 done
'10.1016/j.apcata.2013.09.015'
5578 done
'10.1016/j.electacta.2020.136773'
5579 done
'10.1016/j.apenergy.2012.06.023'
5580 done
'10.1016/j.fuel.2019.116703'
5581 done
'10.1016/j.ijhydene.2008.05.014'
5582 done
'10.1016/j.supflu.2013.11.015'
5583 done
'10.1016/j.apcatb.2021.120220'
5584 done
'10.1016/j.commatsci.2011.1

5759 done
'10.1016/j.jics.2021.100143'
5760 done
'10.1016/j.apenergy.2020.115184'
5761 done
'10.1016/B978-0-12-804568-8.00006-8'
5762 done
'10.1016/j.ijhydene.2017.03.183'
5763 done
'10.1016/j.ijhydene.2008.06.077'
5764 done
'10.1016/j.apcata.2010.11.006'
5765 done
'10.1016/S1381-1169(00)00566-5'
5766 done
'10.1016/j.ijhydene.2008.05.084'
5767 done
'10.1016/j.ijhydene.2017.12.145'
5768 done
'10.1016/j.jcis.2018.08.016'
5769 done
'10.1016/j.compositesb.2019.02.058'
5770 done
'10.1016/j.fluid.2007.06.033'
5771 done
'10.1016/j.fuel.2018.04.107'
5772 done
'10.1016/j.solmat.2020.110813'
5773 done
'10.1016/j.indcrop.2019.05.027'
5774 done
'10.1016/j.cattod.2015.05.008'
5775 done
'10.1016/j.biotechadv.2021.107882'
5776 done
'10.1016/j.apcatb.2020.118873'
5777 done
'10.1016/j.jcat.2020.05.006'
5778 done
'10.1016/j.biortech.2019.122252'
5779 done
'10.1016/j.ccr.2018.06.005'
5780 done
'10.1016/j.apcata.2016.11.033'
5781 done
'10.1016/j.enconman.2020.112859'
5782 done
'10.1016/j.memsci.2019.11719

In [11]:
df_available_abstract = pd.DataFrame()
df_available_abstract['DOI'] = list_doi
df_available_abstract['Title'] = list_title
df_available_abstract['Abstract'] = list_abstract

df_available_abstract.head(10)

,DOI,Title,Abstract
0,10.1016/j.tet.2021.132473,Homogeneous first-row transition metal catalys...,The recent advances in homogeneous first-row t...
1,10.1016/j.apcatb.2020.119868,Selective photoelectrocatalytic tuning of benz...,Hydrogen production can be promoted by replaci...
2,10.1016/j.rser.2020.110523,Light alcohols reforming towards renewable hyd...,Fuel cells (FC) produce electricity in a conti...
3,10.1016/j.apsusc.2021.150446,Minimized Pt deposition on CdS simultaneously ...,Constructing coupling reaction system is an ef...
4,10.1016/j.cej.2021.129870,Boosting photocatalytic hydrogen production co...,Photocatalytic H2 production or photocatalytic...
5,10.1016/B978-0-12-823827-1.00001-8,8 Production of γ-valerolactone and furfuryl a...,\n The production of drop-in che...
6,10.1016/j.mcat.2020.110912,Continuous furfuryl alcohol production via cou...,We have investigated the continuous (P = 1 atm...
7,10.1016/j.jcis.2021.02.036,Modulating oxygen vacancies on bismuth-molybda...,Photocatalytic selective oxidation of alcohols...
8,10.1016/j.apsusc.2021.151284,CdS(ZB)/CdS(WZ)/Ni-BTC photocatalytic selectiv...,Development of difunctional photocatalyst for ...
9,10.1016/j.jelechem.2021.115820,Quinone shuttling impels selective electrocata...,Today development of efficient catalytic syste...


In [12]:
df_available_abstract.to_csv(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\Jupyter_file\Ex_scibert\Abstract\final_abstract.csv')